# Imports & Loading Dataset

In [1]:
import numpy as np
import pandas as pd

## Requirements

In [ ]:
%pip install transformers evaluate langchain

## Upload files to collab

In [ ]:
from google.colab import files
data = files.upload()

# Preprocessing

In [8]:
# data = pd.read_json('./dataset.json')

In [ ]:
print(data.describe())
print(data.info())

In [10]:
def preprocessing(df):
  df['review_text'] = df['review_text'].str.lower()

In [ ]:
preprocessing(data)
data.head()

In [ ]:
from langchain_huggingface.llms import HuggingFacePipeline

from transformers import AlbertForSequenceClassification, AlbertTokenizer, pipeline

model_id = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_id)
model = AlbertForSequenceClassification.from_pretrained(model_id, num_labels=5)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

prompt = "Your objective is to read user reviews for books and determine the final rating given by the user on a scale of 1 to 5 stars. Being 1 the lowest score and 5 the highest."